In [1]:
from utils.experimentor import *
from utils.constants import *  

In [2]:
config = {
    "dataset_name": Dataset.OGBN_ARXIV,
    'num_of_epochs': 2,
    'num_of_runs': 2,
    #'patience_period': 100,
    
    #'should_visualize': False,
    'batch_size': 512,
    'test_batch_size': 1024,
    'num_workers': 6,
    'force_cpu': False,
    #'console_log_freq': 10,
    
    'lr': 0.0001,
    'num_of_layers': 3,  # PPI has got 42% of nodes with all 0 features - that's why 3 layers are useful
    'num_heads': 4,
    #'num_heads_per_layer': [4, 4, 6],  # other values may give even better results from the reported ones
    #'num_features_per_layer': [PPI_NUM_INPUT_FEATURES, 256, 256, PPI_NUM_CLASSES],  # 64 would also give ~0.975 uF1!
    #'add_skip_connection': True,  # skip connection is very important! (keep it otherwise micro-F1 is almost 0)
    #'bias': True,  # bias doesn't matter that much
    #'dropout': 0.0,  # dropout hurts the performance (best to keep it at 0)
    
    #'neighbourhood_degree': 2,
    #'adjacency_mode': AdjacencyMode.OneStep
}


In [3]:
experimentor = Experimentor(config)
experimentor.run()

torch.cuda is available: True
Dataset.OGBN_ARXIV

Run 01:



Epoch 01:  65%|██████▌   | 59392/90941 [00:24<00:02, 13342.42it/s]